# Instructions

For Part A, you need to scrape IMDB web page to find out top movies sorted by user votes. For each movie, you need to pull :
- movie_id
- rank
- title 
- year
- rating
- runtime
- votes
- genres

The URL of an page that include movies released between 2018 and 2020 sorted by number of votes is: 

https://www.imdb.com/search/title/?at=0&sort=num_votes,desc&start=1&title_type=feature&year=2018,2020


Please click the URL and investigate how you can pull movie_id, rank, title,... from the webpage. This webpage, however, only includes 50 movies. Hence, if you want to extract top 250 movies between 2018 and 2020 according to  the number of votes, you need to click “next” 4 times and parse 4 more pages. Fortunately, we can do this by modifying the URL a little. For example, the URL

https://www.imdb.com/search/title/?at=0&sort=num_votes,desc&start=51&title_type=feature&year=2018,2020
(Note that start = 1 in the first URL, now start = 51) allows us to move on to the next page. Obviously, https://www.imdb.com/search/title/?at=0&sort=num_votes,desc&start=101&title_type=feature&year=2018,2020
will lead us to the third page. 


**You need to write code after where I have <span style="color:red">'''  Your code here ...    '''.</span>**

***
Now let’s look at each function in detail. The parameter “top_number” in the function read_m_by_voting(first_year, last_year, top_number)  represents the top number of movies you want to retrieve. For example, read_m_by_voting(2018,2020,500) means that we want to extract top 500 movies released between 2018 and 2020, sorted by users' votes.

This function returns a list of dictionaries. Each dictionary represents one of the top movies, which could look like the following:

{   
   
      'movie_id': 'tt6324278',
      'title': 'Abominable',
      'year': '(2019)',
      'rank': '358.', 
      'runtime': '97 min', 
      'rating': '7.0', 
      'votes': '34,093'
      'genres': 'Animation, Adventure, Comedy'
}

In order to implement this function read_m_by_voting(first_year, last_year, top_number), you need to first implement the function read_m_from_url(url, num_of_m=50). This read_m_from_url function is used to extract num_of_m number of movies from a URL. It will also return a list of dictionaries, each of which represents a movie. As described above, to extract say top 120 movies, you need to parse 3 webpage because each page includes only 50 movies. This read_m_from_url function allows you to extract a specific number of movies from a URL. 

After you implement “read_m_by_voting”, which will return a list of top movies, you need to implement the function write_movies_csv(final_list, filename) to write the movies list to a csv file.

***
You probably want to first work on read_m_from_url, then read_m_by_voting, and then write_movies_csv. For each of the functions, I included a test function. For instance, for the function, read_m_from_url, I have included a test function called test_read_m_from_url(). Please un-comment the test function to test your code. The test function test_write_movies_csv () outputs "IMDb_TopVoted.csv". You need to test each function before moving onto the next. Even within a function, you may need to use print() to test your code very carefully.

***

After you done with scraping the needed data, you should clean and transform it as needed to make it ready for enriching the given "Movies.csv" dataset.

Finaly, export the enriched dataset to a CSV file:
Use the following naming convention: Project_3_PartA_Group#.csv



### Helper functions
***

The helper functions section includes some functions you need to use when implementing the three functions describe above. This inculdes the implementation of two functions and test function for each helper function: read_html(url), test_read_html(), process_str_with_comma(string), and test_process_str_with_comma(). For instance, running  test_process_str_with_comma() will help you understand what the function process_str_with_comma(string) does.



In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import math
import warnings
import re
import numpy as np
warnings.filterwarnings('ignore')

# Helper functions:

In [2]:
"""
The read_html(url) return the contents of an URL hmtl file as a string.
"""
def read_html(url):
    response = requests.get(url)
    content = response.content
    return content

#===========================================================================

"""
If a string contains comma, use "" to enclose the string to write it to a comma-separated values (CSV) 
file with no issue with the comma.
"""
def process_str_with_comma(string):
    if ',' in string:
        new_string = '"' + string.strip() + '"'
    else:
        new_string = string
    return new_string


### Test  read_html 

In [3]:
def test_read_html():
    print (read_html('https://www.imdb.com/search/title/?at=0&sort=num_votes,desc&start=1&title_type=feature&year=2018,2020'))


In [4]:
test_read_html()

b'\n\n<!DOCTYPE html>\n<html\n    xmlns:og="http://ogp.me/ns#"\n    xmlns:fb="http://www.facebook.com/2008/fbml">\n    <head>\n         \n\n        <meta charset="utf-8">\n        <meta http-equiv="X-UA-Compatible" content="IE=edge">\n\n\n\n\n        <script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:\'java\'};</script>\n\n<script>\n    if (typeof uet == \'function\') {\n      uet("bb", "LoadTitle", {wb: 1});\n    }\n</script>\n  <script>(function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);</script>\n        <title>Feature Film,\nReleased between 2018-01-01 and 2020-12-31\n(Sorted by Number of Votes Descending) - IMDb</title>\n  <script>(function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);</script>\n<script>\n    if (typeof uet == \'function\') {\n      uet("be", "LoadTitle", {wb: 1});\n    }\n</script>\n<script>\n    if (typeof uex == \'function\') {\n      

### Test  process_str_with_comma 

In [5]:
def test_process_str_with_comma():
    """output:
    string: it is a string
    string: "it is a string, right"
    """
    string = 'it is a string'
    print ("string: " + process_str_with_comma(string))
    string = 'it is a string, right'
    print ("string: " + process_str_with_comma(string))


In [6]:
test_process_str_with_comma()

string: it is a string
string: "it is a string, right"


***

## read_m_from_url

Inside this function, you need to write your code to pull the movies information.
For each movie, you need to pull :
- movie_id
- title 
- rank
- year
- runtime
- rating
- votes 
- genres

To give examples on how to pull data from web bage, I have included the code to pull the movie_id.
You need to inculde your code to pull the other needed movie information (title, rank, year, ......). You should have no missing values for each of the collected data.

In [8]:
def read_m_from_url(url, num_of_m=50):
    print(url)
    # this function, read a number of movies from a url. The default value is 50
    
    html_string = read_html(url) # given a url you need to read the hmtl file as a string. 
    # I have included  the read_html function in the helper functions. Please take a look.
    
    # create a soup object
    soup = BeautifulSoup(html_string, "html.parser")
    
    '''
    Click the URL and investigate how you can pull movie_id, rank, title,... from the webpage.
    
    To investigate the html of a web page , For example:
    URL: https://www.imdb.com/search/title/?at=0&sort=num_votes,desc&start=1&title_type=feature&year=2018,2020
    Right-click anywhere on the webpage, and at the very bottom of the menu that pops up, 
    you will see "Inspect", Click on it.
    '''
    
# Fetching a div that includes all the movies. This can be done by using find and find_all functions.
    # for example, find_all('div') will give you all divs on the page. Actually, 
    # this find or find_all function can have two parameters,
    # in the code below 'div' is the tag name and 'lister-list' is an attribute value of the tag. You can also do
    # movie_list = soup.find('div', 'lister-list'). Here you explicitly say: I want to find a div with 
    # attribute class = 'lister-list'.
    
    # Since on each imdb page, there's only one div with class = 'lister-list', we can use find rather than find_all. 
    # Find_all will return a list of div tags, while find() will return only one div.
    
    movie_list = soup.find('div', 'lister-list') # this div contains all the listed movies in 
                                                 # the requested html web page.
    
    list_movies = [] # initialize the function return value, which is a list of movies. 
                     # This list will contains the scraped data transformed to a structured format.
    
    # Using count track the number of movies processed. now it's 0 - No movie has been processed yet.
    count = 0
    
    # each movie listed in a div with attribute value 'lister-item mode-advanced'.
    divs= movie_list.find_all('div','lister-item mode-advanced') # To find all the listed movies in the page.
    for d in divs:
        dict_each_movie = {}  # initialize the movie dictionary to store the movie information.

        # Pulling the movie_id
        try:
            h = d.find('h3','lister-item-header') 
            movie_id= h.find('a').attrs['href']
            movie_id= movie_id[7:-1]
            
        except:
            print("Error in pulling movie_id")
        finally:
            dict_each_movie["movie_id"] = movie_id
            print(movie_id)

        # Pulling the title
        '''  Your code here ...    '''
        try:
            movie_title = d.find('span','lister-item-index unbold text-primary').find_next().text
            
        except:
            print("Error in pulling titles")
        finally:
            dict_each_movie["movie_title"] = movie_title
            print(movie_title)   
           
        # Pulling the rank
        '''  Your code here ...    '''
        try:
            movie_rank = d.find('span','lister-item-index unbold text-primary').text
            
        except:
            print("Error in pulling rank")
        finally:
            dict_each_movie["movie_rank"] = movie_rank
            print(movie_rank) 
        
        
        # Pulling the year
        '''  Your code here ...    '''
        try:
            movie_year = d.find('span','lister-item-year text-muted unbold').text
            
        except:
            print("Error in pulling year")
        finally:
            if ' ' in movie_year:
                movie_year = movie_year.split(' ')
                movie_year = movie_year[1]
            dict_each_movie["movie_year"] = movie_year
            print(movie_year)


        # Pulling the runtime
        '''  Your code here ...    '''
        try:
            movie_runtime = d.find('span','runtime').text
            
        except:
            print("Error in pulling runtime")
        finally:
            dict_each_movie["movie_runtime"] = movie_runtime
            print(movie_runtime)

        
        # Pulling the rating
        '''  Your code here ...    '''
        try:
            movie_rating = d.find('div','inline-block ratings-imdb-rating').get('data-value')
            
        except:
            print("Error in pulling rating")
        finally:
            dict_each_movie["movie_rating"] = movie_rating
            print(movie_rating)
                     

        # Pulling the votes
        '''  Your code here ...    '''
        try:
            movie_votes = d.find('p','sort-num_votes-visible').find_all('span')[1].text
            
        except:
            print("Error in pulling votes")
        finally:
            dict_each_movie["movie_votes"] = movie_votes
            print(movie_votes)

        
        # Pulling the genres
        '''  Your code here ...    '''
        try:
            movie_genres = d.find('span','genre').text.strip()
            
        except:
            print("Error in pulling genres")
        finally:
            dict_each_movie["movie_genres"] = movie_genres
            print(movie_genres)

            
        list_movies.append(dict_each_movie)  # To add the movie information to the movies list.

        count +=1
        print('===============================')
        print()
        if count == num_of_m:
            break # to exit from the loop.

    return list_movies


### Test  read_m_from_url

In [11]:

def test_read_m_from_url():
    """ output:
    Movies list: [{'movie_id': 'tt7286456', 'title': 'Joker', 'year': '(2019)', 'rank': '1.', 'runtime': '122 min', 'rating': '8.4', 'votes': '1,074,230'}, {'movie_id': 'tt4154796', 'title': 'Avengers: Endgame', 'year': '(2019)', 'rank': '2.', 'runtime': '181 min', 'rating': '8.4', 'votes': '945,461'}, {'movie_id': 'tt4154756', 'title': 'Avengers: Infinity War', 'year': '(2018)', 'rank': '3.', 'runtime': '149 min', 'rating': '8.4', 'votes': '928,596'}, {'movie_id': 'tt1825683', 'title': 'Black Panther', 'year': '(2018)', 'rank': '4.', 'runtime': '134 min', 'rating': '7.3', 'votes': '678,964'}, {'movie_id': 'tt6751668', 'title': 'Parasite', 'year': '(2019)', 'rank': '5.', 'runtime': '132 min', 'rating': '8.6', 'votes': '666,646'}, {'movie_id': 'tt7131622', 'title': 'Once Upon a Time... In Hollywood', 'year': '(2019)', 'rank': '6.', 'runtime': '161 min', 'rating': '7.6', 'votes': '642,048'}, {'movie_id': 'tt8946378', 'title': 'Knives Out', 'year': '(2019)', 'rank': '7.', 'runtime': '130 min', 'rating': '7.9', 'votes': '534,299'}, {'movie_id': 'tt5463162', 'title': 'Deadpool 2', 'year': '(2018)', 'rank': '8.', 'runtime': '119 min', 'rating': '7.7', 'votes': '519,760'}, {'movie_id': 'tt8579674', 'title': '1917', 'year': '(2019)', 'rank': '9.', 'runtime': '119 min', 'rating': '8.3', 'votes': '495,380'}, {'movie_id': 'tt4154664', 'title': 'Captain Marvel', 'year': '(2019)', 'rank': '10.', 'runtime': '123 min', 'rating': '6.8', 'votes': '493,817'}, {'movie_id': 'tt1727824', 'title': 'Bohemian Rhapsody', 'year': '(2018)', 'rank': '11.', 'runtime': '134 min', 'rating': '7.9', 'votes': '489,064'}, {'movie_id': 'tt6644200', 'title': 'A Quiet Place', 'year': '(2018)', 'rank': '12.', 'runtime': '90 min', 'rating': '7.5', 'votes': '482,141'}, {'movie_id': 'tt4633694', 'title': 'Spider-Man: Into the Spider-Verse', 'year': '(2018)', 'rank': '13.', 'runtime': '117 min', 'rating': '8.4', 'votes': '430,153'}, {'movie_id': 'tt6966692', 'title': 'Green Book', 'year': '(2018)', 'rank': '14.', 'runtime': '130 min', 'rating': '8.2', 'votes': '428,762'}, {'movie_id': 'tt6723592', 'title': 'Tenet', 'year': '(2020)', 'rank': '15.', 'runtime': '150 min', 'rating': '7.4', 'votes': '426,125'}, {'movie_id': 'tt1477834', 'title': 'Aquaman', 'year': '(2018)', 'rank': '16.', 'runtime': '143 min', 'rating': '6.9', 'votes': '417,286'}, {'movie_id': 'tt1270797', 'title': 'Venom', 'year': '(2018)', 'rank': '17.', 'runtime': '112 min', 'rating': '6.7', 'votes': '410,565'}, {'movie_id': 'tt2527338', 'title': 'Star Wars: The Rise Of Skywalker', 'year': '(2019)', 'rank': '18.', 'runtime': '141 min', 'rating': '6.5', 'votes': '404,527'}, {'movie_id': 'tt1677720', 'title': 'Ready Player One', 'year': '(2018)', 'rank': '19.', 'runtime': '140 min', 'rating': '7.4', 'votes': '398,599'}, {'movie_id': 'tt6320628', 'title': 'Spider-Man: Far from Home', 'year': '(2019)', 'rank': '20.', 'runtime': '129 min', 'rating': '7.4', 'votes': '383,087'}, {'movie_id': 'tt1517451', 'title': 'A Star Is Born', 'year': '(2018)', 'rank': '21.', 'runtime': '136 min', 'rating': '7.6', 'votes': '356,745'}, {'movie_id': 'tt1302006', 'title': 'The Irishman', 'year': '(2019)', 'rank': '22.', 'runtime': '209 min', 'rating': '7.8', 'votes': '352,691'}, {'movie_id': 'tt5095030', 'title': 'Ant-Man and the Wasp', 'year': '(2018)', 'rank': '23.', 'runtime': '118 min', 'rating': '7.0', 'votes': '346,822'}, {'movie_id': 'tt2584384', 'title': 'Jojo Rabbit', 'year': '(2019)', 'rank': '24.', 'runtime': '108 min', 'rating': '7.9', 'votes': '340,733'}, {'movie_id': 'tt1950186', 'title': 'Ford v Ferrari', 'year': '(2019)', 'rank': '25.', 'runtime': '152 min', 'rating': '8.1', 'votes': '337,490'}, {'movie_id': 'tt3778644', 'title': 'Solo: A Star Wars Story', 'year': '(2018)', 'rank': '26.', 'runtime': '135 min', 'rating': '6.9', 'votes': '313,683'}, {'movie_id': 'tt4912910', 'title': 'Mission: Impossible - Fallout', 'year': '(2018)', 'rank': '27.', 'runtime': '147 min', 'rating': '7.7', 'votes': '308,761'}, {'movie_id': 'tt6146586', 'title': 'John Wick: Chapter 3 - Parabellum', 'year': '(2019)', 'rank': '28.', 'runtime': '130 min', 'rating': '7.4', 'votes': '306,871'}, {'movie_id': 'tt2737304', 'title': 'Bird Box', 'year': '(2018)', 'rank': '29.', 'runtime': '124 min', 'rating': '6.6', 'votes': '305,135'}, {'movie_id': 'tt2798920', 'title': 'Annihilation', 'year': '(I) (2018)', 'rank': '30.', 'runtime': '115 min', 'rating': '6.8', 'votes': '298,808'}, {'movie_id': 'tt0448115', 'title': 'Shazam!', 'year': '(2019)', 'rank': '31.', 'runtime': '132 min', 'rating': '7.0', 'votes': '295,848'}, {'movie_id': 'tt4881806', 'title': 'Jurassic World: Fallen Kingdom', 'year': '(2018)', 'rank': '32.', 'runtime': '128 min', 'rating': '6.2', 'votes': '283,862'}, {'movie_id': 'tt8367814', 'title': 'The Gentlemen', 'year': '(2019)', 'rank': '33.', 'runtime': '113 min', 'rating': '7.8', 'votes': '282,900'}, {'movie_id': 'tt2948372', 'title': 'Soul', 'year': '(2020)', 'rank': '34.', 'runtime': '100 min', 'rating': '8.1', 'votes': '280,267'}, {'movie_id': 'tt7653254', 'title': 'Marriage Story', 'year': '(2019)', 'rank': '35.', 'runtime': '137 min', 'rating': '7.9', 'votes': '274,033'}, {'movie_id': 'tt7784604', 'title': 'Hereditary', 'year': '(2018)', 'rank': '36.', 'runtime': '127 min', 'rating': '7.3', 'votes': '270,157'}, {'movie_id': 'tt3606756', 'title': 'Incredibles 2', 'year': '(2018)', 'rank': '37.', 'runtime': '118 min', 'rating': '7.6', 'votes': '269,810'}, {'movie_id': 'tt6857112', 'title': 'Us', 'year': '(II) (2019)', 'rank': '38.', 'runtime': '116 min', 'rating': '6.8', 'votes': '257,673'}, {'movie_id': 'tt8772262', 'title': 'Midsommar', 'year': '(2019)', 'rank': '39.', 'runtime': '148 min', 'rating': '7.1', 'votes': '255,303'}, {'movie_id': 'tt0437086', 'title': 'Alita: Battle Angel', 'year': '(2019)', 'rank': '40.', 'runtime': '122 min', 'rating': '7.3', 'votes': '247,522'}, {'movie_id': 'tt5727208', 'title': 'Uncut Gems', 'year': '(2019)', 'rank': '41.', 'runtime': '135 min', 'rating': '7.4', 'votes': '246,741'}, {'movie_id': 'tt6139732', 'title': 'Aladdin', 'year': '(2019)', 'rank': '42.', 'runtime': '128 min', 'rating': '6.9', 'votes': '245,331'}, {'movie_id': 'tt7349662', 'title': 'BlacKkKlansman', 'year': '(2018)', 'rank': '43.', 'runtime': '135 min', 'rating': '7.5', 'votes': '242,325'}, {'movie_id': 'tt4123430', 'title': 'Fantastic Beasts: The Crimes of Grindelwald', 'year': '(2018)', 'rank': '44.', 'runtime': '134 min', 'rating': '6.5', 'votes': '239,448'}, {'movie_id': 'tt7126948', 'title': 'Wonder Woman 1984', 'year': '(2020)', 'rank': '45.', 'runtime': '151 min', 'rating': '5.4', 'votes': '232,962'}, {'movie_id': 'tt7349950', 'title': 'It Chapter Two', 'year': '(2019)', 'rank': '46.', 'runtime': '169 min', 'rating': '6.5', 'votes': '230,874'}, {'movie_id': 'tt6105098', 'title': 'The Lion King', 'year': '(2019)', 'rank': '47.', 'runtime': '118 min', 'rating': '6.8', 'votes': '226,997'}, {'movie_id': 'tt6823368', 'title': 'Glass', 'year': '(2019)', 'rank': '48.', 'runtime': '129 min', 'rating': '6.6', 'votes': '224,677'}, {'movie_id': 'tt1979376', 'title': 'Toy Story 4', 'year': '(2019)', 'rank': '49.', 'runtime': '100 min', 'rating': '7.7', 'votes': '223,650'}, {'movie_id': 'tt2704998', 'title': 'Game Night', 'year': '(I) (2018)', 'rank': '50.', 'runtime': '100 min', 'rating': '6.9', 'votes': '220,159'}]    
    """
    url = "http://www.imdb.com/search/title?at=0&sort=num_votes,desc&start=1&title_type=feature&year=2018,2020"
    print ("Movies list: ", read_m_from_url(url))

In [12]:
 test_read_m_from_url()

http://www.imdb.com/search/title?at=0&sort=num_votes,desc&start=1&title_type=feature&year=2018,2020
tt7286456
Joker
1.
(2019)
122 min
8.4
1,181,805
Crime, Drama, Thriller

tt4154796
Avengers: Endgame
2.
(2019)
181 min
8.4
1,041,926
Action, Adventure, Drama

tt4154756
Avengers: Infinity War
3.
(2018)
149 min
8.5
1,005,682
Action, Adventure, Sci-Fi

tt6751668
Parasite
4.
(2019)
132 min
8.5
737,328
Comedy, Drama, Horror

tt1825683
Black Panther
5.
(2018)
134 min
7.3
717,526
Action, Adventure, Sci-Fi

tt7131622
Once Upon a Time... In Hollywood
6.
(2019)
161 min
7.6
684,202
Comedy, Drama

tt8946378
Knives Out
7.
(2019)
130 min
7.9
586,284
Comedy, Crime, Drama

tt5463162
Deadpool 2
8.
(2018)
119 min
7.7
550,715
Action, Adventure, Comedy

tt8579674
1917
9.
(2019)
119 min
8.3
545,807
Action, Drama, War

tt4154664
Captain Marvel
10.
(2019)
123 min
6.8
529,545
Action, Adventure, Sci-Fi

tt1727824
Bohemian Rhapsody
11.
(2018)
134 min
7.9
514,687
Biography, Drama, Music

tt6644200
A Quiet Place
12

***
##  read_m_by_voting

This method takes two years (first_year, last_year) and a number (top_number) as inputs. 
If we have first year = 2018, and last_year =2020, and top_number = 500, we want to retrieve top 500 movies 
that were released between 2018 and 2020, we need to do two things:

##### 1. Construct a url. An example url can be 
    https://www.imdb.com/search/title/?at=0&sort=num_votes,desc&start=1&title_type=feature&year=2018,2020
    
        This URL means that the web page will display movies based on user_voting in descending 
        order "sort=num_votes,desc" between year 2018 and 2020,
        It will start from the first movie (i.e., start=1 - we need to set the start index in the url).
    
        IMDB will display just 50 movies based on this URL. So in order to review more 
        movies, we need to click "next" on the web page. By clicking the next button,
        we will see a new url
           https://www.imdb.com/search/title/?at=0&sort=num_votes,desc&start=51&title_type=feature&year=2018,2020

        If we compare the two urls above, we can easily see that in the second url, start=51, i.e., IMDB provides 
        another 50 movies, starting from movie No 51.If we want to retrieve the top 61 movies, we need to open 
        two web pages with two urls. 
        And if we want to retrieve top 256 movies, we need to open 6 different URLs.
        Obviously, we want to use a loop to construct the different URLs.
    
    
##### 2. Read movies from the URL: using "read_m_from_url" method. 
      What this method does is that it opens a url and read numbers_of_movies_you_need_to_read_on_the_page. 
      For example, if we want to read top 61 movies, we will need to first open 
      a url https://www.imdb.com/search/title/?at=0&sort=num_votes,desc&start=1&title_type=feature&year=2018,2020
      
      We need to read all 50 movies on the page by calling read_m_from_url(url), and then we need to open 
      the second url http://www.imdb.com/search/title?at=0&sort=user_rating&start=1&title_type=feature&year=2005,2016, but 
      we just need to read 61-50=11 movies from the page. Now, let's set current_index = 51 and top_number = 61, we 
      actually need to retrieve (top_number - current_index + 1) movies. we can do this by calling
      read_m_from_url(url, top_number - the current_index+1).
      
      Hence, we need to use if statement here. Based on top_number, we may need to open multiple urls 
         (e.g., if top 256, we need to open 6 urls)
    

In [13]:
m_per_page = 50 # by default, imdb return 50 movies page url.
def read_m_by_voting(first_year, last_year, top_number):
    
    current_index = 1  # initialize current_index. In the first iteration, we need to have start = 1.
    
    final_list = []  # initialize the return value. This method returns a list. Each item in the list is a dictionary. 
                     # Each dictionary includes information regarding a movie.

    for i in range(int(math.ceil(top_number/50.0))):
        url= 'http://www.imdb.com/search/title/?at=0&sort=num_votes,desc&start='+str(current_index)+'&title_type=feature&year='+str(first_year)+','+ str(last_year)

        if (i+1) == len(range( int(math.ceil(top_number/50.0)))):
            lis = read_m_from_url(url, top_number - current_index + 1)
        else:
            lis = read_m_from_url(url, m_per_page)
        final_list += lis
        current_index +=50

    return final_list

### Test read_m_by_voting

In [14]:
def test_read_m_by_voting():
    print (read_m_by_voting(2018,2020,500))  # print a list of top 500 movies sorted by user number of votes

In [15]:
test_read_m_by_voting()

http://www.imdb.com/search/title/?at=0&sort=num_votes,desc&start=1&title_type=feature&year=2018,2020
tt7286456
Joker
1.
(2019)
122 min
8.4
1,181,805
Crime, Drama, Thriller

tt4154796
Avengers: Endgame
2.
(2019)
181 min
8.4
1,041,926
Action, Adventure, Drama

tt4154756
Avengers: Infinity War
3.
(2018)
149 min
8.5
1,005,682
Action, Adventure, Sci-Fi

tt6751668
Parasite
4.
(2019)
132 min
8.5
737,328
Comedy, Drama, Horror

tt1825683
Black Panther
5.
(2018)
134 min
7.3
717,526
Action, Adventure, Sci-Fi

tt7131622
Once Upon a Time... In Hollywood
6.
(2019)
161 min
7.6
684,202
Comedy, Drama

tt8946378
Knives Out
7.
(2019)
130 min
7.9
586,284
Comedy, Crime, Drama

tt5463162
Deadpool 2
8.
(2018)
119 min
7.7
550,715
Action, Adventure, Comedy

tt8579674
1917
9.
(2019)
119 min
8.3
545,807
Action, Drama, War

tt4154664
Captain Marvel
10.
(2019)
123 min
6.8
529,545
Action, Adventure, Sci-Fi

tt1727824
Bohemian Rhapsody
11.
(2018)
134 min
7.9
514,687
Biography, Drama, Music

tt6644200
A Quiet Place
1

tt6266538
Vice
101.
(2018)
132 min
7.2
141,343
Biography, Comedy, Drama

tt4729430
Klaus
102.
(2019)
96 min
8.1
141,110
Animation, Adventure, Comedy

tt7146812
Onward
103.
(2020)
102 min
7.4
140,424
Animation, Adventure, Comedy

tt7798634
Ready or Not
104.
(2019)
95 min
6.8
139,942
Action, Comedy, Horror

tt13143964
Borat Subsequent Moviefilm
105.
(2020)
95 min
6.6
136,301
Comedy

tt5814060
The Nun
106.
(2018)
96 min
5.3
136,038
Horror, Mystery, Thriller

tt10272386
The Father
107.
(2020)
97 min
8.3
133,163
Drama, Mystery

tt4500922
Maze Runner: The Death Cure
108.
(2018)
143 min
6.2
132,974
Action, Adventure, Sci-Fi

tt7959026
The Mule
109.
(2018)
116 min
7
132,531
Crime, Drama, Thriller

tt2283336
Men in Black: International
110.
(2019)
114 min
5.6
129,965
Action, Adventure, Comedy

tt2854926
Tag
111.
(2018)
100 min
6.5
129,887
Action, Comedy

tt8110330
Dil Bechara
112.
(2020)
101 min
8.1
128,393
Comedy, Drama, Romance

tt9893250
I Care a Lot
113.
(2020)
118 min
6.3
127,465
Comedy, C

tt8350360
Annabelle Comes Home
201.
(2019)
106 min
5.9
72,197
Horror, Mystery, Thriller

tt4003440
The House That Jack Built
202.
(2018)
152 min
6.8
72,173
Crime, Drama, Horror

tt6921996
Johnny English Strikes Again
203.
(2018)
89 min
6.2
72,170
Action, Adventure, Comedy

tt4532826
Robin Hood
204.
(2018)
116 min
5.3
72,140
Action, Adventure, Drama

tt1226837
Beautiful Boy
205.
(2018)
120 min
7.3
71,332
Biography, Drama

tt2990140
The Christmas Chronicles
206.
(2018)
104 min
7
71,325
Adventure, Comedy, Family

tt1137450
Death Wish
207.
(2018)
107 min
6.3
71,272
Action, Crime, Drama

tt10618286
Mank
208.
(2020)
131 min
6.8
71,215
Biography, Comedy, Drama

tt8155288
Happy Death Day 2U
209.
(2019)
100 min
6.2
70,997
Comedy, Horror, Mystery

tt5220122
Hotel Transylvania 3: Summer Vacation
210.
(2018)
97 min
6.3
70,890
Animation, Adventure, Comedy

tt4332232
Fractured
211.
(2019)
99 min
6.4
70,610
Thriller

tt5033998
Charlie's Angels
212.
(2019)
118 min
4.9
70,368
Action, Adventure, Comedy


tt5177088
The Girl in the Spider's Web
301.
(2018)
115 min
6.1
46,973
Action, Crime, Drama

tt6079516
I See You
302.
(2019)
98 min
6.8
46,874
Crime, Drama, Horror

tt5116302
Togo
303.
(2019)
113 min
7.9
46,772
Adventure, Biography, Drama

tt10431500
Miracle in Cell No. 7
304.
(2019)
132 min
8.2
46,769
Drama

tt5814534
Spies in Disguise
305.
(2019)
102 min
6.8
46,426
Animation, Action, Adventure

tt7734218
Stuber
306.
(2019)
93 min
6.1
46,422
Action, Comedy, Crime

tt7431594
Race 3
307.
(2018)
160 min
2.1
45,949
Action, Crime, Thriller

tt5431890
Official Secrets
308.
(2019)
112 min
7.3
45,764
Biography, Crime, Drama

tt7153766
Unsane
309.
(2018)
98 min
6.4
45,495
Drama, Horror, Mystery

tt5073642
Color Out of Space
310.
(2019)
111 min
6.1
45,042
Horror, Mystery, Sci-Fi

tt2837574
The Old Man & the Gun
311.
(2018)
93 min
6.7
45,042
Biography, Comedy, Crime

tt6491178
Dragged Across Concrete
312.
(2018)
159 min
6.9
44,822
Action, Crime, Drama

tt1289403
The Guernsey Literary and Potato P

tt10324144
Article 15
401.
(2019)
130 min
8.2
32,320
Crime, Drama, Mystery

tt4761916
Unfriended: Dark Web
402.
(2018)
92 min
6
32,073
Horror, Mystery, Thriller

tt8108268
The Tashkent Files
403.
(2019)
134 min
8.5
31,988
Drama, Mystery, Thriller

tt9806192
I Lost My Body
404.
(2019)
81 min
7.5
31,809
Animation, Drama, Fantasy

tt3833480
The Outpost
405.
(2019)
123 min
6.8
31,693
Action, Drama, History

tt4073790
The Darkest Minds
406.
(2018)
104 min
5.7
31,688
Action, Adventure, Drama

tt9398640
Between Two Ferns: The Movie
407.
(2019)
82 min
6.1
31,482
Comedy

tt8633462
Quo Vadis, Aida?
408.
(2020)
101 min
8.1
31,473
Drama, History, War

tt6513656
Escape Plan 2: Hades
409.
(2018)
96 min
3.8
31,323
Action, Crime, Mystery

tt8983202
Kabir Singh
410.
(2019)
173 min
7.1
31,276
Action, Drama, Romance

tt5935704
Padmaavat
411.
(2018)
164 min
7.1
31,254
Drama, History, Romance

tt10612922
One Night in Miami...
412.
(2020)
114 min
7.1
31,189
Drama

tt8850222
Peninsula
413.
(2020)
116 min
5.5

***
# write_movies_csv
This method to write the movies to a csv file.
Each row in the csv file represents a movie. 
 - The parameter final_list includes a number of movies, which is the output of the function read_m_by_voting. 
 - The filename represents the output file name.
 
 ***
 Important note:  make sure to complete the code of read_m_from_url method before running write_movies_csv method.
 ***

In [16]:

def write_movies_csv(final_list, filename):

    lis = [] # to write the file, we create a list of strings.
    
    header = "movie_id,"+"movie_rank,"+"movie_title,"+"movie_year,"+"movie_rating,"+"movie_runtime,"+"movie_votes,"+"movie_genres"
    lis.append(header) # add the header to the list
    for movie in final_list:
        string =  process_str_with_comma(movie["movie_id"]) +"," + process_str_with_comma(movie["movie_rank"]) +","+ process_str_with_comma(movie["movie_title"]) +","+ process_str_with_comma(movie["movie_year"]) + ","+  process_str_with_comma(movie["movie_rating"]) +","+ process_str_with_comma(movie["movie_runtime"]) + "," + process_str_with_comma(movie["movie_votes"]) + "," + process_str_with_comma(movie["movie_genres"])
        print (string)
        lis.append(string)# add the string to the list

    # Writing the strings list to csv file
    f = None
    f = open(filename,"w")
    for s in lis:
        f.write("%s\n" % s)
    f.close()
    
    return


In [17]:
# The output of the test_write_movies_csv method is the "IMDb_TopVoted.csv" file.
def test_write_movies_csv(): 
    li = read_m_by_voting(2018, 2020, 500) # To read the top voted 500 movies between 2018 and 2020 from imdb.
    print(li)
    print("================================================================")
    write_movies_csv(li,"IMDb_TopVoted.csv") 


In [18]:
test_write_movies_csv()

http://www.imdb.com/search/title/?at=0&sort=num_votes,desc&start=1&title_type=feature&year=2018,2020
tt7286456
Joker
1.
(2019)
122 min
8.4
1,181,805
Crime, Drama, Thriller

tt4154796
Avengers: Endgame
2.
(2019)
181 min
8.4
1,041,926
Action, Adventure, Drama

tt4154756
Avengers: Infinity War
3.
(2018)
149 min
8.5
1,005,682
Action, Adventure, Sci-Fi

tt6751668
Parasite
4.
(2019)
132 min
8.5
737,328
Comedy, Drama, Horror

tt1825683
Black Panther
5.
(2018)
134 min
7.3
717,526
Action, Adventure, Sci-Fi

tt7131622
Once Upon a Time... In Hollywood
6.
(2019)
161 min
7.6
684,202
Comedy, Drama

tt8946378
Knives Out
7.
(2019)
130 min
7.9
586,284
Comedy, Crime, Drama

tt5463162
Deadpool 2
8.
(2018)
119 min
7.7
550,715
Action, Adventure, Comedy

tt8579674
1917
9.
(2019)
119 min
8.3
545,807
Action, Drama, War

tt4154664
Captain Marvel
10.
(2019)
123 min
6.8
529,545
Action, Adventure, Sci-Fi

tt1727824
Bohemian Rhapsody
11.
(2018)
134 min
7.9
514,687
Biography, Drama, Music

tt6644200
A Quiet Place
1

tt6266538
Vice
101.
(2018)
132 min
7.2
141,343
Biography, Comedy, Drama

tt4729430
Klaus
102.
(2019)
96 min
8.1
141,110
Animation, Adventure, Comedy

tt7146812
Onward
103.
(2020)
102 min
7.4
140,424
Animation, Adventure, Comedy

tt7798634
Ready or Not
104.
(2019)
95 min
6.8
139,942
Action, Comedy, Horror

tt13143964
Borat Subsequent Moviefilm
105.
(2020)
95 min
6.6
136,301
Comedy

tt5814060
The Nun
106.
(2018)
96 min
5.3
136,038
Horror, Mystery, Thriller

tt10272386
The Father
107.
(2020)
97 min
8.3
133,163
Drama, Mystery

tt4500922
Maze Runner: The Death Cure
108.
(2018)
143 min
6.2
132,974
Action, Adventure, Sci-Fi

tt7959026
The Mule
109.
(2018)
116 min
7
132,531
Crime, Drama, Thriller

tt2283336
Men in Black: International
110.
(2019)
114 min
5.6
129,965
Action, Adventure, Comedy

tt2854926
Tag
111.
(2018)
100 min
6.5
129,887
Action, Comedy

tt8110330
Dil Bechara
112.
(2020)
101 min
8.1
128,393
Comedy, Drama, Romance

tt9893250
I Care a Lot
113.
(2020)
118 min
6.3
127,465
Comedy, C

tt8350360
Annabelle Comes Home
201.
(2019)
106 min
5.9
72,197
Horror, Mystery, Thriller

tt4003440
The House That Jack Built
202.
(2018)
152 min
6.8
72,173
Crime, Drama, Horror

tt6921996
Johnny English Strikes Again
203.
(2018)
89 min
6.2
72,170
Action, Adventure, Comedy

tt4532826
Robin Hood
204.
(2018)
116 min
5.3
72,140
Action, Adventure, Drama

tt1226837
Beautiful Boy
205.
(2018)
120 min
7.3
71,332
Biography, Drama

tt2990140
The Christmas Chronicles
206.
(2018)
104 min
7
71,325
Adventure, Comedy, Family

tt1137450
Death Wish
207.
(2018)
107 min
6.3
71,272
Action, Crime, Drama

tt10618286
Mank
208.
(2020)
131 min
6.8
71,215
Biography, Comedy, Drama

tt8155288
Happy Death Day 2U
209.
(2019)
100 min
6.2
70,997
Comedy, Horror, Mystery

tt5220122
Hotel Transylvania 3: Summer Vacation
210.
(2018)
97 min
6.3
70,890
Animation, Adventure, Comedy

tt4332232
Fractured
211.
(2019)
99 min
6.4
70,610
Thriller

tt5033998
Charlie's Angels
212.
(2019)
118 min
4.9
70,368
Action, Adventure, Comedy


tt5177088
The Girl in the Spider's Web
301.
(2018)
115 min
6.1
46,973
Action, Crime, Drama

tt6079516
I See You
302.
(2019)
98 min
6.8
46,874
Crime, Drama, Horror

tt5116302
Togo
303.
(2019)
113 min
7.9
46,772
Adventure, Biography, Drama

tt10431500
Miracle in Cell No. 7
304.
(2019)
132 min
8.2
46,769
Drama

tt5814534
Spies in Disguise
305.
(2019)
102 min
6.8
46,426
Animation, Action, Adventure

tt7734218
Stuber
306.
(2019)
93 min
6.1
46,422
Action, Comedy, Crime

tt7431594
Race 3
307.
(2018)
160 min
2.1
45,949
Action, Crime, Thriller

tt5431890
Official Secrets
308.
(2019)
112 min
7.3
45,764
Biography, Crime, Drama

tt7153766
Unsane
309.
(2018)
98 min
6.4
45,495
Drama, Horror, Mystery

tt5073642
Color Out of Space
310.
(2019)
111 min
6.1
45,042
Horror, Mystery, Sci-Fi

tt2837574
The Old Man & the Gun
311.
(2018)
93 min
6.7
45,042
Biography, Comedy, Crime

tt6491178
Dragged Across Concrete
312.
(2018)
159 min
6.9
44,822
Action, Crime, Drama

tt1289403
The Guernsey Literary and Potato P

tt10324144
Article 15
401.
(2019)
130 min
8.2
32,320
Crime, Drama, Mystery

tt4761916
Unfriended: Dark Web
402.
(2018)
92 min
6
32,073
Horror, Mystery, Thriller

tt8108268
The Tashkent Files
403.
(2019)
134 min
8.5
31,988
Drama, Mystery, Thriller

tt9806192
I Lost My Body
404.
(2019)
81 min
7.5
31,809
Animation, Drama, Fantasy

tt3833480
The Outpost
405.
(2019)
123 min
6.8
31,693
Action, Drama, History

tt4073790
The Darkest Minds
406.
(2018)
104 min
5.7
31,688
Action, Adventure, Drama

tt9398640
Between Two Ferns: The Movie
407.
(2019)
82 min
6.1
31,482
Comedy

tt8633462
Quo Vadis, Aida?
408.
(2020)
101 min
8.1
31,473
Drama, History, War

tt6513656
Escape Plan 2: Hades
409.
(2018)
96 min
3.8
31,323
Action, Crime, Mystery

tt8983202
Kabir Singh
410.
(2019)
173 min
7.1
31,276
Action, Drama, Romance

tt5935704
Padmaavat
411.
(2018)
164 min
7.1
31,254
Drama, History, Romance

tt10612922
One Night in Miami...
412.
(2020)
114 min
7.1
31,189
Drama

tt8850222
Peninsula
413.
(2020)
116 min
5.5

# Importing the given dataset "Movies.csv" to Pandas DataFrame called df1

In [19]:
# Importing the csv file to df1 and print the df1.
'''  Your code here ...    '''
df1 = pd.read_csv("~/Desktop/Movies.csv")
df1

movie_id titleType           originalTitle  isAdult
0     tt7286456     movie                   Joker        0
1     tt4154796     movie       Avengers: Endgame        0
2     tt4154756     movie  Avengers: Infinity War        0
3     tt6751668     movie            Gisaengchung        0
4     tt1825683     movie           Black Panther        0
..          ...       ...                     ...      ...
495   tt8266310     movie    Blinded by the Light        0
496   tt5316540     movie                   Close        0
497   tt9340860     movie              Let Him Go        0
498  tt10199590     movie          Les misérables        0
499   tt6628102     movie             Ahlat Agaci        0

[500 rows x 4 columns]

# Import the scraped data from the IMDb_TopVoted.csv file to Pandas DataFrame called df2

In [20]:
# You need to import the collected dataset "IMDb_TopVoted.csv".
# To handel Latin characters that may contained in the csv file
# with no issue, use  encoding= "ISO-8859-1" with the pd.read_csv()
# Example: df1 = pd.read_csv('thefilename.csv', encoding= "ISO-8859-1") 
# Using encoding= "ISO-8859-1" will avoid Unicode-Decode-Errors.
'''  Your code here ...    '''
df2 = pd.read_csv("~/Desktop/IMDb_TopVoted.csv", encoding = "ISO-8859-1")
df2

movie_id  movie_rank             movie_title movie_year  movie_rating  \
0     tt7286456         1.0                   Joker     (2019)           8.4   
1     tt4154796         2.0       Avengers: Endgame     (2019)           8.4   
2     tt4154756         3.0  Avengers: Infinity War     (2018)           8.5   
3     tt6751668         4.0                Parasite     (2019)           8.5   
4     tt1825683         5.0           Black Panther     (2018)           7.3   
..          ...         ...                     ...        ...           ...   
495   tt8266310       496.0    Blinded by the Light     (2019)           6.9   
496   tt5316540       497.0                   Close     (2019)           5.7   
497   tt9340860       498.0              Let Him Go     (2020)           6.7   
498  tt10199590       499.0         Les MisÃ©rables     (2019)           7.6   
499   tt8652728       500.0                   Waves     (2019)           7.5   

    movie_runtime movie_votes               movie_genres  
0         122 min   1,181,805     Crime, Drama, Thriller  
1         181 min   1,041,926   Action, Adventure, Drama  
2         149 min   1,005,682  Action, Adventure, Sci-Fi  
3         132 min     737,328      Comedy, Drama, Horror  
4         134 min     717,526  Action, Adventure, Sci-Fi  
..            ...         ...                        ...  
495       118 min      24,506       Comedy, Drama, Music  
496        94 min      24,359    Action, Drama, Thriller  
497       113 min      24,297     Crime, Drama, Thriller  
498       104 min      24,149     Crime, Drama, Thriller  
499       135 min      24,064      Drama, Romance, Sport  

[500 rows x 8 columns]

## Data cleansing and transformation for df2.

In [21]:
# Cleaning and tranforming df2
 # rank, year, runtime, and votes should have a numeric integer data type.
'''  Your code here ...    '''
cols = df2.columns
for col in cols:
    unique = df2[col].unique()
    print(col,'\n',len(unique), unique, '\n**********************************************************************')

movie_id 
 500 ['tt7286456' 'tt4154796' 'tt4154756' 'tt6751668' 'tt1825683' 'tt7131622'
 'tt8946378' 'tt5463162' 'tt8579674' 'tt4154664' 'tt1727824' 'tt6644200'
 'tt4633694' 'tt6723592' 'tt1270797' 'tt6966692' 'tt6320628' 'tt1477834'
 'tt2527338' 'tt1677720' 'tt5095030' 'tt1517451' 'tt1302006' 'tt2584384'
 'tt1950186' 'tt3778644' 'tt6146586' 'tt2737304' 'tt4912910' 'tt0448115'
 'tt2798920' 'tt8367814' 'tt2948372' 'tt4881806' 'tt7653254' 'tt7784604'
 'tt3606756' 'tt8772262' 'tt6857112' 'tt5727208' 'tt4123430' 'tt0437086'
 'tt6139732' 'tt7349662' 'tt7126948' 'tt7349950' 'tt1979376' 'tt7975244'
 'tt6105098' 'tt6823368' 'tt2704998' 'tt7713068' 'tt2935510' 'tt9243946'
 'tt1051906' 'tt5164214' 'tt1365519' 'tt8228288' 'tt8332922' 'tt6806448'
 'tt7984734' 'tt5083738' 'tt8936646' 'tt3281548' 'tt1213641' 'tt2873282'
 'tt5606664' 'tt6499752' 'tt6565702' 'tt3741700' 'tt1560220' 'tt6450804'
 'tt1070874' 'tt2066051' 'tt8106534' 'tt4779682' 'tt3104988' 'tt4520988'
 'tt5104604' 'tt2231461' 'tt7846844'

In [22]:
df2 = df2.convert_dtypes()
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   movie_id       500 non-null    string 
 1   movie_rank     500 non-null    Int64  
 2   movie_title    500 non-null    string 
 3   movie_year     500 non-null    string 
 4   movie_rating   500 non-null    Float64
 5   movie_runtime  500 non-null    string 
 6   movie_votes    500 non-null    string 
 7   movie_genres   500 non-null    string 
dtypes: Float64(1), Int64(1), string(6)
memory usage: 32.4 KB


In [23]:
# Remove parentheses in movie_year column
df2['movie_year'] = df2['movie_year'].str.replace('(','').str.replace(')','')
df2

movie_id  movie_rank             movie_title movie_year  movie_rating  \
0     tt7286456           1                   Joker       2019           8.4   
1     tt4154796           2       Avengers: Endgame       2019           8.4   
2     tt4154756           3  Avengers: Infinity War       2018           8.5   
3     tt6751668           4                Parasite       2019           8.5   
4     tt1825683           5           Black Panther       2018           7.3   
..          ...         ...                     ...        ...           ...   
495   tt8266310         496    Blinded by the Light       2019           6.9   
496   tt5316540         497                   Close       2019           5.7   
497   tt9340860         498              Let Him Go       2020           6.7   
498  tt10199590         499         Les MisÃ©rables       2019           7.6   
499   tt8652728         500                   Waves       2019           7.5   

    movie_runtime movie_votes               movie_genres  
0         122 min   1,181,805     Crime, Drama, Thriller  
1         181 min   1,041,926   Action, Adventure, Drama  
2         149 min   1,005,682  Action, Adventure, Sci-Fi  
3         132 min     737,328      Comedy, Drama, Horror  
4         134 min     717,526  Action, Adventure, Sci-Fi  
..            ...         ...                        ...  
495       118 min      24,506       Comedy, Drama, Music  
496        94 min      24,359    Action, Drama, Thriller  
497       113 min      24,297     Crime, Drama, Thriller  
498       104 min      24,149     Crime, Drama, Thriller  
499       135 min      24,064      Drama, Romance, Sport  

[500 rows x 8 columns]

In [24]:
df2.isnull().sum()

movie_id         0
movie_rank       0
movie_title      0
movie_year       0
movie_rating     0
movie_runtime    0
movie_votes      0
movie_genres     0
dtype: int64

In [25]:
# Handling the 2 string columns
df2['movie_votes'] = df2['movie_votes'].str.replace(",","").astype('Int64')
df2['movie_runtime'] = df2['movie_runtime'].str.replace("min","").astype('Int64')

df2['movie_rank'] = df2['movie_rank'].astype('Int64')
df2['movie_year'] = df2['movie_year'].astype('Int64')

In [27]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   movie_id       500 non-null    string 
 1   movie_rank     500 non-null    Int64  
 2   movie_title    500 non-null    string 
 3   movie_year     500 non-null    Int64  
 4   movie_rating   500 non-null    Float64
 5   movie_runtime  500 non-null    Int64  
 6   movie_votes    500 non-null    Int64  
 7   movie_genres   500 non-null    string 
dtypes: Float64(1), Int64(4), string(3)
memory usage: 33.8 KB


# 	Enrich the given dataset (df1) by merging it to the scraped data (df2).

In [28]:
# Merege the two dataframes to one dataframe called df.
'''  Your code here ...    '''
df = pd.merge(df1, df2, on=['movie_id'])
df

movie_id titleType              originalTitle  isAdult  movie_rank  \
0     tt7286456     movie                      Joker        0           1   
1     tt4154796     movie          Avengers: Endgame        0           2   
2     tt4154756     movie     Avengers: Infinity War        0           3   
3     tt6751668     movie               Gisaengchung        0           4   
4     tt1825683     movie              Black Panther        0           5   
..          ...       ...                        ...      ...         ...   
494   tt6521876     movie  Jay and Silent Bob Reboot        0         495   
495   tt8266310     movie       Blinded by the Light        0         496   
496   tt5316540     movie                      Close        0         497   
497   tt9340860     movie                 Let Him Go        0         498   
498  tt10199590     movie             Les misérables        0         499   

                   movie_title  movie_year  movie_rating  movie_runtime  \
0                        Joker        2019           8.4            122   
1            Avengers: Endgame        2019           8.4            181   
2       Avengers: Infinity War        2018           8.5            149   
3                     Parasite        2019           8.5            132   
4                Black Panther        2018           7.3            134   
..                         ...         ...           ...            ...   
494  Jay and Silent Bob Reboot        2019           5.6            105   
495       Blinded by the Light        2019           6.9            118   
496                      Close        2019           5.7             94   
497                 Let Him Go        2020           6.7            113   
498            Les MisÃ©rables        2019           7.6            104   

     movie_votes               movie_genres  
0        1181805     Crime, Drama, Thriller  
1        1041926   Action, Adventure, Drama  
2        1005682  Action, Adventure, Sci-Fi  
3         737328      Comedy, Drama, Horror  
4         717526  Action, Adventure, Sci-Fi  
..           ...                        ...  
494        24743  Action, Adventure, Comedy  
495        24506       Comedy, Drama, Music  
496        24359    Action, Drama, Thriller  
497        24297     Crime, Drama, Thriller  
498        24149     Crime, Drama, Thriller  

[499 rows x 11 columns]

### Rearrange the dataset fields to be listed in the following order: 
 movie_id, rank, votes, title, originalTitle, year, rating, titleType, isAdult, runtime,  genres

In [29]:
# Rearrange the dataset fields.
'''  Your code here ...    '''
df = df[['movie_id', 'movie_rank', 'movie_votes', 'movie_title', 'originalTitle', 'movie_year', 'movie_rating',
        'titleType','isAdult', 'movie_runtime','movie_genres']]


# Export the enriched dataset to a CSV file:

In [30]:
# Use the following naming convention: 
#  Project_3_PartA_Group#.csv

'''  Your code here ...    '''
df.to_csv('Project_3_PartA_Group#16.csv', index =False, encoding= "cp1252")